In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv("data.csv", low_memory=False)
edit = data.copy()

In [3]:
data.head(5)

,nombre_inmueble,aviso,precio,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
0,CASAS-ALQUILER-TEMPORAL-PUNTA-COLORADA-034,16579724,U$S 1.750,600 Mts,Casa,2 dormitorios,Av. Primera entre Rambla Costanera y 8 Guazubi...,Punta Colorada,Alquiler temporario,2021-01-03 00:00:00
1,"Alquilo Apto. 2 Dorm. Vista , 2 da Enero y Mar...",16735528,$U 800,NaN,Apartamento,2 dormitorios,Mi hijo el doctor y En Familia,Punta Del Este,Alquiler temporario,2021-01-03 00:00:00
2,"Casa en Cerro del Burro ""MIRADOR II""",13795860,U$S 100.000,300 Mts,Casa,1 dormitorio,... y Cerro del burro,Piriápolis,Venta,2021-01-03 00:00:00
3,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,U$S 1.600,NaN,Oficina,NaN,... y Buceo,Buceo,Alquiler,2021-01-03 00:00:00
4,Terreno en Punta Colorada,12805785,U$S 67.000,615 Mts,Terreno,NaN,TE201282 y Punta colorada,Punta Colorada,Venta,2021-01-03 00:00:00


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96583 entries, 0 to 96582
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_inmueble  96462 non-null  object
 1   aviso            96583 non-null  object
 2   precio           96462 non-null  object
 3   superficie       81049 non-null  object
 4   tipo             96462 non-null  object
 5   dormitorios      75070 non-null  object
 6   direccion        96128 non-null  object
 7   barrio           96462 non-null  object
 8   negocio          96462 non-null  object
 9   fecha            96583 non-null  object
dtypes: object(10)
memory usage: 7.4+ MB


Clasificación por:
    - tipo de finca
    - tipo de negocio: afecta el precio (alquileres mensuales)

In [5]:
data["negocio"].value_counts()

Venta                  66165
Alquiler               15553
Alquiler temporario    14744
Name: negocio, dtype: int64

In [6]:
data["tipo"].value_counts()

Apartamento    45098
Casa           29972
Terreno        11076
Oficina         4455
Local           3944
Campo           1375
Comercio         202
Edificio         157
Garage           111
Piezas            66
Otro               6
Name: tipo, dtype: int64

In [7]:
data["direccion"].isna().sum(), data["direccion"].count() #Conteo de direcciones

(455, 96128)

In [8]:
data["barrio"].isna().sum(), data["barrio"].count() #Conteo de barrio, posibilidad de imputar direcciones faltantes

(121, 96462)

In [9]:
data["precio"].isna().sum()
#Si el estudio es sobre oportunidad de inversión, estos datos deben eliminarse

121

# Filtrado por municipio
### Se identificaron varias filas pertenecientes a localidades fuera de las solicitadas por consigna.

In [10]:
municipios = {"B": ["Aguada", "parte de La Comercial", "Tres Cruces", "Cordón", "Parque Rodó", "Palermo", "Sur", "Ciudad Vieja", "Centro"],
"E": ["Unión","Malvín Norte","Malvín Nuevo","Las Canteras","Carrasco Norte","Carrasco","Punta Gorda","Malvín","Buceo","La Blanqueada"],
"C": ["Prado", "Aires Puros", "Atahualpa", "Nueva Savona", "Brazo Oriental", "Mercado Modelo", "Bolívar", "Larrañaga", "Jacinto Vera", "La Figurita", "La Comercial", "Villa Muñoz", "Retiro", "Goes", "Aguada", "una parte del Puerto", "Capurro", "Bella Vista", "Reducto"],
"CH": ["La Blanqueada", "Parque Batlle", "Villa Dolores", "Pocitos", "Punta Carretas", "Tres Cruces", "Larrañaga", "Buceo"]}

In [11]:
barrios_ok = []
for value in list(municipios.values()):
    barrios_ok.extend(value)

In [12]:
for index in edit.index:
    if edit["barrio"][index] not in barrios_ok:
        edit.drop(index, axis=0, inplace=True)
# Esta celda demora en correr 10-15 minutos: Recorre y checkea los barrios de todo el dataset.

In [13]:
edit.drop_duplicates("nombre_inmueble")
edit.reset_index(drop=True, inplace=True)
#Eliminación de avisos repetidos, reseteo de índice

# Limpieza de precio 

In [14]:
edit["precio"].isna().sum() #No hay valores faltantes

0

In [15]:
edit['precio'] = edit['precio'].replace('\$','', regex = True)
uyu = edit['precio'].str.contains('U ', case=True, regex=True)
UYUloc = uyu[uyu==True].index.values
edit['precio'][UYUloc] = edit['precio'].iloc[UYUloc].str.replace('U','')

In [16]:
UYUloc

array([    2,     7,    16, ..., 32275, 32277, 32280], dtype=int64)

In [17]:
edit['precio'] = edit['precio'].str.replace('US ','')
edit['precio'] = edit['precio'].str.replace('.','', regex = True)
edit['precio'] = edit['precio'].astype(str).astype(int).astype(float)

Las siguientes dos líneas de código no fueron utilizadas ya que correr el código toma mucho tiempo. Lo dejamos a modo demostrativo pero la conversión fue realizada por valor constante.

In [ ]:
import yfinance as yahooFinance
from datetime import datetime, timedelta

In [ ]:
def convert_currency(src, dst, amount):
    # construct the currency pair symbol
    symbol = f"{src}{dst}=X"
    FX = yahooFinance.Ticker(symbol)
    
    # extract minute data of the recent 2 days
    latest_data = FX.history(period = '1d')
    
    # get the latest datetime
    last_updated_datetime = latest_data.index[-1].to_pydatetime()
    
    # get the latest price
    latest_price = latest_data['Close'].iloc[-1]
    
    # return the latest datetime with the converted amount
    return latest_price * amount

In [18]:
edit.rename(columns = {'precio':'precio U$S'}, inplace = True)
edit["precio U$S"][UYUloc] = edit["precio U$S"][UYUloc].apply(lambda x: x/40)
#No correr más de una vez, opera por cada vez que se corre

In [19]:
edit.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
0,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,1600.0,NaN,Oficina,NaN,... y Buceo,Buceo,Alquiler,2021-01-03 00:00:00
1,Oficina Sosa - Local Con Vivienda Alquilado En...,16224275,125000.0,NaN,Local,NaN,Oficina Sosa - Local Con Vivienda Alquilado En...,Brazo Oriental,Venta,2021-01-03 00:00:00
2,Excelente Local Amplio - Gran Sótano Centro,16436745,625.0,NaN,Local,NaN,Soriano y Julio Herrera Y Obes,Centro,Alquiler,2021-01-03 00:00:00
3,Terreno en el Centro,16703708,220000.0,442 Mts,Terreno,NaN,Wilson Ferreira Aldunate y Durazno,Centro,Venta,2021-01-03 00:00:00
4,Impecable apartamento en zona privilegiada.,12452693,380000.0,130 Mts,Apartamento,3 dormitorios,Ramon Fernandez y Bulevar Artigas,Punta Carretas,Venta,2021-01-03 00:00:00


In [ ]:
import plotly.express as px
for value in edit["negocio"].unique():
    fig = px.box(edit[edit["negocio"]==value], x="tipo", y="precio U$S", title=("Negocio: "+value))
    fig.show()

In [ ]:
plot_data = edit[edit['tipo'].isin(edit['tipo'].unique())]
plot_data = plot_data[['tipo', 'precio U$S']]
#plot_data = plot_data[plot_data['tipo']]
plot_data.boxplot(by='tipo', grid=True, figsize=(10,10), 
                  showfliers=False #le quite todos los puntos para que se visualize mejor, si ponemos True se ven los outliers
                 )

plt.show()

# Manejo de columna superficie

In [21]:
def hta_finder(df,column):
    hta = []
    for index in range(len(df[column])):
        if type(df[column].iloc[index]) != float and df[column].iloc[index].find("Has") != -1:
            hta.append(index)
    return hta

In [22]:
edit_hta = edit.iloc[hta_finder(edit,"superficie")]

In [23]:
edit.drop(edit_hta[edit_hta["superficie"] == "1244 Has."].index, axis=0, inplace=True)
#Se checkearon los valores y habían quedado avisos de San José de Mayo, fueron eliminados a mano

In [24]:
edit.drop(edit[(edit["barrio"]=="Bella Vista") & (edit["direccion"].str.contains("y Bella vista"))].index, axis=0, inplace=True)
#Encontramos que había fincas del balneario Bella Vista y no del barrio de Montevideo

In [25]:
edit.reset_index(drop=True, inplace=True)
#Después de borrar valores, se debe resetear el index

In [27]:
#Notamos que en superficie, habían valores en Has. que no coincidian con el nombre del inmueble y por la ubicación, 
#era imposible que terrenos de tal dimensión estuvieran disponibles. Fueron transformados a m2.
edit_hta = edit.iloc[hta_finder(edit,"superficie")]
index_hta = edit_hta.index
edit["superficie"][index_hta] = edit["superficie"][index_hta].str.replace(" Has.","", regex=True)
edit["superficie"][index_hta] = edit["superficie"][index_hta].astype(str).astype(int).astype(float)

In [28]:
edit['superficie'] = edit['superficie'].str.replace(' Mts','')
edit['superficie'] = pd.to_numeric(edit['superficie'], errors='coerce')
# Se pasan todos los valores a m2

In [29]:
edit["superficie"].dtype
#Todos los valores quedaron como float

dtype('float64')

In [30]:
edit.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,dormitorios,direccion,barrio,negocio,fecha
0,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,1600.0,NaN,Oficina,NaN,... y Buceo,Buceo,Alquiler,2021-01-03 00:00:00
1,Oficina Sosa - Local Con Vivienda Alquilado En...,16224275,125000.0,NaN,Local,NaN,Oficina Sosa - Local Con Vivienda Alquilado En...,Brazo Oriental,Venta,2021-01-03 00:00:00
2,Excelente Local Amplio - Gran Sótano Centro,16436745,625.0,NaN,Local,NaN,Soriano y Julio Herrera Y Obes,Centro,Alquiler,2021-01-03 00:00:00
3,Terreno en el Centro,16703708,220000.0,442.0,Terreno,NaN,Wilson Ferreira Aldunate y Durazno,Centro,Venta,2021-01-03 00:00:00
4,Impecable apartamento en zona privilegiada.,12452693,380000.0,130.0,Apartamento,3 dormitorios,Ramon Fernandez y Bulevar Artigas,Punta Carretas,Venta,2021-01-03 00:00:00


# Creación de coordenadas por barrio y manejo de direcciones

In [ ]:
import geopandas
from geopy.geocoders import Nominatim

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
barrios = edit["barrio"].value_counts().index
coords = [("{},{}").format(locator.geocode(barrio+",Montevideo, Uruguay").latitude, locator.geocode(barrio+",Montevideo, Uruguay").longitude) for barrio in barrios] 

coords_t = [coord.split(",") for coord in coords]
coords_2 = []
for coord in coords_t:
    for card in coord:
        coords_2.append(float(card))
final = lambda coords_2, x: [coords_2[i:i+2] for i in range(0, len(coords_2), 2)]
coords_final = final(coords_2, 2)

barrios_coords = dict(zip(barrios,coords_final))

#### Se adquieren las coordenadas de los barrios para plotear y visualizar en mapa qué tanto se repiten las localidades.

In [ ]:
# histograma barrios
#heatmap

### Se elimina la columna dirección ya que no agrega valor al análisis, en varios casos incompleto o "misleading". Además, el valor de las propiedades tiende a regirse más por el barrio donde se ubica que por la calle en particular.

In [31]:
edit.drop(labels="direccion",axis=1, inplace=True)

In [32]:
edit.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,dormitorios,barrio,negocio,fecha
0,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,1600.0,NaN,Oficina,NaN,Buceo,Alquiler,2021-01-03 00:00:00
1,Oficina Sosa - Local Con Vivienda Alquilado En...,16224275,125000.0,NaN,Local,NaN,Brazo Oriental,Venta,2021-01-03 00:00:00
2,Excelente Local Amplio - Gran Sótano Centro,16436745,625.0,NaN,Local,NaN,Centro,Alquiler,2021-01-03 00:00:00
3,Terreno en el Centro,16703708,220000.0,442.0,Terreno,NaN,Centro,Venta,2021-01-03 00:00:00
4,Impecable apartamento en zona privilegiada.,12452693,380000.0,130.0,Apartamento,3 dormitorios,Punta Carretas,Venta,2021-01-03 00:00:00


In [33]:
edit.drop_duplicates("aviso", inplace=True)
#Se eliminaron los avisos duplicados

In [34]:
len(edit) #Se redujo significativamente el tamaño del dataset

19682

In [35]:
edit.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,dormitorios,barrio,negocio,fecha
0,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,1600.0,NaN,Oficina,NaN,Buceo,Alquiler,2021-01-03 00:00:00
1,Oficina Sosa - Local Con Vivienda Alquilado En...,16224275,125000.0,NaN,Local,NaN,Brazo Oriental,Venta,2021-01-03 00:00:00
2,Excelente Local Amplio - Gran Sótano Centro,16436745,625.0,NaN,Local,NaN,Centro,Alquiler,2021-01-03 00:00:00
3,Terreno en el Centro,16703708,220000.0,442.0,Terreno,NaN,Centro,Venta,2021-01-03 00:00:00
4,Impecable apartamento en zona privilegiada.,12452693,380000.0,130.0,Apartamento,3 dormitorios,Punta Carretas,Venta,2021-01-03 00:00:00


# Limpieza y corrección de dormitorios

### En este paso, debemos separar el dataset en dos:
#### Por un lado, para aquellas fincas donde la cantidad de dormitorios es relevante (casas y apartamentos). df = civil
#### Por otro, para aquellas que no es relevante tal dato y es imposible de imputar (Ej: Oficina) df = com

In [36]:
dic_dorms = {
    0:0,
    "Monoambiente":1,
    "1 dormitorio":1,
    "2 dormitorios":2,
    "3 dormitorios":3,
    "4 dormitorios":4,
    "más de 4 dormitorios":5,}
#Diccionario para mapear valores para casas y apartamentos

In [47]:
civil = edit[(edit["tipo"]=="Apartamento") | (edit["tipo"]=="Casa")]
civil.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,dormitorios,barrio,negocio,fecha
4,Impecable apartamento en zona privilegiada.,12452693,380000.0,130.0,Apartamento,3 dormitorios,Punta Carretas,Venta,2021-01-03 00:00:00
5,Venta apartamento ciudad vieja 2 dormitorios,16660467,129000.0,75.0,Apartamento,2 dormitorios,Ciudad Vieja,Venta,2021-01-03 00:00:00
6,Próximo al Parque y Rambla! OPORTUNIDAD!,13853448,115000.0,30.0,Apartamento,Monoambiente,Punta Carretas,Venta,2021-01-03 00:00:00
9,Exclusiva Ideal 2 familias Empresa Centro de ...,15032833,570000.0,NaN,Casa,más de 4 dormitorios,Punta Gorda,Venta,2021-01-03 00:00:00
10,"Hermosa Casa, Con Todas Las Comodidades!!!",16609637,380000.0,249.0,Casa,4 dormitorios,Prado,Venta,2021-01-03 00:00:00


In [49]:
civil["dormitorios"] = civil["dormitorios"].map(dic_dorms)
#Para transformar de string a int

In [55]:
civil["dormitorios"].isna().sum() #No hay valores faltantes

0

In [56]:
civil["dormitorios"].value_counts()

1    5547
2    5502
3    3772
4    1015
5     725
Name: dormitorios, dtype: int64

In [48]:
com = edit[(edit["tipo"]!="Apartamento") & (edit["tipo"]!="Casa")]
com.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,dormitorios,barrio,negocio,fecha
0,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,1600.0,NaN,Oficina,NaN,Buceo,Alquiler,2021-01-03 00:00:00
1,Oficina Sosa - Local Con Vivienda Alquilado En...,16224275,125000.0,NaN,Local,NaN,Brazo Oriental,Venta,2021-01-03 00:00:00
2,Excelente Local Amplio - Gran Sótano Centro,16436745,625.0,NaN,Local,NaN,Centro,Alquiler,2021-01-03 00:00:00
3,Terreno en el Centro,16703708,220000.0,442.0,Terreno,NaN,Centro,Venta,2021-01-03 00:00:00
7,Ellauri Y Montero,16613348,1200.0,NaN,Local,NaN,Punta Carretas,Alquiler,2021-01-03 00:00:00


In [52]:
com["dormitorios"].isna().sum(), len(com) #Todos los valores de este atributo son nulos

(3121, 3121)

In [53]:
com.drop(labels="dormitorios",axis=1, inplace=True)
#Se elimina la columna dormitorios

In [54]:
com.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,barrio,negocio,fecha
0,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,1600.0,NaN,Oficina,Buceo,Alquiler,2021-01-03 00:00:00
1,Oficina Sosa - Local Con Vivienda Alquilado En...,16224275,125000.0,NaN,Local,Brazo Oriental,Venta,2021-01-03 00:00:00
2,Excelente Local Amplio - Gran Sótano Centro,16436745,625.0,NaN,Local,Centro,Alquiler,2021-01-03 00:00:00
3,Terreno en el Centro,16703708,220000.0,442.0,Terreno,Centro,Venta,2021-01-03 00:00:00
7,Ellauri Y Montero,16613348,1200.0,NaN,Local,Punta Carretas,Alquiler,2021-01-03 00:00:00


# Mapeo de negocio y tipo: One Hot Encoding

### Para poder introducir el dataset a un modelo de Machine Learning, debe estar en un formato legible. Tratamos estas dos variables categóricas con este método para convertir esta información en variables numéricas.

In [58]:
civil.drop(index= civil[civil['negocio'].isna()==True].index.values, inplace=True)
com.drop(index= com[com['negocio'].isna()==True].index.values, inplace=True)
#Se eliminan las celdas con valores faltantes en negocio

In [60]:
#Primero observamos si hay valores faltantes
civil["barrio"].isna().sum(), civil["negocio"].isna().sum(), civil["tipo"].isna().sum(), com["barrio"].isna().sum(), com["negocio"].isna().sum(), com["tipo"].isna().sum()
#No hay valores faltantes

(0, 0, 0, 0, 0, 0)

In [61]:
# Al haber borrado datos, se deben reajustar los índices nuevamente.
civil.reset_index(drop=True, inplace=True)
com.reset_index(drop=True, inplace=True)

In [62]:
civil.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,dormitorios,barrio,negocio,fecha
0,Impecable apartamento en zona privilegiada.,12452693,380000.0,130.0,Apartamento,3,Punta Carretas,Venta,2021-01-03 00:00:00
1,Venta apartamento ciudad vieja 2 dormitorios,16660467,129000.0,75.0,Apartamento,2,Ciudad Vieja,Venta,2021-01-03 00:00:00
2,Próximo al Parque y Rambla! OPORTUNIDAD!,13853448,115000.0,30.0,Apartamento,1,Punta Carretas,Venta,2021-01-03 00:00:00
3,Exclusiva Ideal 2 familias Empresa Centro de ...,15032833,570000.0,NaN,Casa,5,Punta Gorda,Venta,2021-01-03 00:00:00
4,"Hermosa Casa, Con Todas Las Comodidades!!!",16609637,380000.0,249.0,Casa,4,Prado,Venta,2021-01-03 00:00:00


In [63]:
com.head(5)

,nombre_inmueble,aviso,precio U$S,superficie,tipo,barrio,negocio,fecha
0,Oficina Alquiler World Trade Center -WTC- Poci...,15655688,1600.0,NaN,Oficina,Buceo,Alquiler,2021-01-03 00:00:00
1,Oficina Sosa - Local Con Vivienda Alquilado En...,16224275,125000.0,NaN,Local,Brazo Oriental,Venta,2021-01-03 00:00:00
2,Excelente Local Amplio - Gran Sótano Centro,16436745,625.0,NaN,Local,Centro,Alquiler,2021-01-03 00:00:00
3,Terreno en el Centro,16703708,220000.0,442.0,Terreno,Centro,Venta,2021-01-03 00:00:00
4,Ellauri Y Montero,16613348,1200.0,NaN,Local,Punta Carretas,Alquiler,2021-01-03 00:00:00


### Para empezar a crear nuestro modelo, debemos modificar el dataset, manteniendo solo las columnas relevantes para el mismo: Civil
#### Precio, superficie, dormitorios, tipo, barrio y negocio. 3 variables ordinales y 3 categóricas, respectivamente.

In [96]:
df_1 = civil.copy()

In [97]:
df_1.drop(labels=["nombre_inmueble","aviso","fecha"], axis=1, inplace=True)

In [308]:
df_1.loc[df_1["superficie"]<21,"superficie"] = np.nan

In [311]:
df_2 = df_1[df_1["superficie"].isna()==False]

In [312]:
dummies_c = pd.get_dummies(df_2[["barrio","tipo","negocio"]])
dummies_c.head(5)
#Dummies de las variables categóricas

,barrio_Aguada,barrio_Aires Puros,barrio_Atahualpa,barrio_Bella Vista,barrio_Brazo Oriental,barrio_Buceo,barrio_Capurro,barrio_Carrasco,barrio_Carrasco Norte,barrio_Centro,...,barrio_Punta Gorda,barrio_Reducto,barrio_Tres Cruces,barrio_Villa Dolores,barrio_Villa Muñoz,tipo_Apartamento,tipo_Casa,negocio_Alquiler,negocio_Alquiler temporario,negocio_Venta
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1


In [313]:
df_model_1 = pd.concat([df_2,dummies_c],axis=1)
df_model_1

,precio U$S,superficie,tipo,dormitorios,barrio,negocio,barrio_Aguada,barrio_Aires Puros,barrio_Atahualpa,barrio_Bella Vista,...,barrio_Punta Gorda,barrio_Reducto,barrio_Tres Cruces,barrio_Villa Dolores,barrio_Villa Muñoz,tipo_Apartamento,tipo_Casa,negocio_Alquiler,negocio_Alquiler temporario,negocio_Venta
0,380000.0,130.0,Apartamento,3,Punta Carretas,Venta,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,129000.0,75.0,Apartamento,2,Ciudad Vieja,Venta,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,115000.0,30.0,Apartamento,1,Punta Carretas,Venta,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,380000.0,249.0,Casa,4,Prado,Venta,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
5,198900.0,86.0,Apartamento,2,Centro,Venta,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16551,475.0,65.0,Apartamento,2,Centro,Alquiler,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
16552,2300.0,420.0,Apartamento,3,Carrasco,Alquiler,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
16554,130000.0,43.0,Apartamento,2,Pocitos,Venta,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
16556,181000.0,97.0,Apartamento,1,Pocitos,Venta,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


### Solo nos resta imputar los valores de la superficie, para poder proceder a la proyección de mercado.
#### Decidimos no eliminar los valores nulos de la variable superficie, ya que son aproximadamente un 25% de los datos que nos quedan después de hacer toda la limpieza.

In [314]:
# Para el modelo, buscamos el dataset sin valores nulos de superficie y le restamos esta columna: X
# Como y, nos quedamos con la columna superficie
X = df_model_1[df_model_1["superficie"].isna()==False].drop(["tipo","barrio","negocio"],axis=1)
y = df_model_1[df_model_1["superficie"].isna()==False][["superficie"]]

In [315]:
#Para el modelo, se deben estandarizar los datos de precio y superficie.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [316]:
precio_2d = np.array(X["precio U$S"]).reshape(-1,1)
superficie_2d = np.array(y.values).reshape(-1,1)
dorms_2d = np.array(X["dormitorios"]).reshape(-1,1)

In [317]:
X["precio U$S"] = scaler.fit_transform(precio_2d)
X["superficie"] = scaler.fit_transform(superficie_2d)
X["dormitorios"] = scaler.fit_transform(dorms_2d)
y["superficie"] = scaler.fit_transform(superficie_2d)

In [319]:
X.drop(labels="superficie",axis=1, inplace=True)

In [320]:
X.head(5)

,precio U$S,dormitorios,barrio_Aguada,barrio_Aires Puros,barrio_Atahualpa,barrio_Bella Vista,barrio_Brazo Oriental,barrio_Buceo,barrio_Capurro,barrio_Carrasco,...,barrio_Punta Gorda,barrio_Reducto,barrio_Tres Cruces,barrio_Villa Dolores,barrio_Villa Muñoz,tipo_Apartamento,tipo_Casa,negocio_Alquiler,negocio_Alquiler temporario,negocio_Venta
0,0.707472,0.783205,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,-0.291271,-0.134755,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,-0.346978,-1.052714,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,0.707472,1.701164,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
5,-0.013135,-0.134755,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [321]:
y.head(5)

,superficie
0,-0.045260
1,-0.121951
2,-0.184698
4,0.120672
5,-0.106613


In [322]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [326]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_train,y_train)

0.09925155129750884

In [324]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=10)

cross_val_score(LinearRegression(), X, y, cv=cv)
#Observamos que mediante regresión lineal, el modelo no se ajusta a los datos, dando scores < 0.30,
#independientemente del orden que los datos sean dados.

array([0.01704959, 0.08890549, 0.00813468, 0.15935801, 0.10228463])

#### Usaremos GridSearchCV para buscar un modelo que se adapte mejor a nuestros datos.

In [297]:
from sklearn.model_selection import GridSearchCV

from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors

In [298]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
            }
        },
        'Kneighbors_regression' : {
            'model':  neighbors.KNeighborsRegressor(),
            'params': { 
                "n_neighbors":[3,5,7], 
                "weights":["uniform","distance"]
            }
        },
        'supp_vector_machine' : {
            'model': svm.SVR(),
            'params': {
                "kernel":["linear","poly","rbf"]
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['squared_error','friedman_mse'],
                'splitter': ['best','random']
            }
        },
        'Random_Forest': {
            'model': RandomForestRegressor(),
            'params':{
                'max_depth' : np.array([3, 5, 7, 9,11,13,15,17,19])
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        if algo_name == "Random_Forest" or algo_name == 'supp_vector_machine':
            gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
            gs.fit(X,y.values.ravel())
        else:
            gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
            gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
        
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [325]:
find_best_model_using_gridsearchcv(X,y)

,model,best_score,best_params
0,linear_regression,0.049439,{}
1,Kneighbors_regression,-0.051181,"{'n_neighbors': 7, 'weights': 'uniform'}"
2,supp_vector_machine,0.067155,{'kernel': 'rbf'}
3,decision_tree,-0.719271,"{'criterion': 'friedman_mse', 'splitter': 'best'}"
4,Random_Forest,0.047701,{'max_depth': 3}


#### Observamos que con ningún modelo, pudimos obtener un score suficientemente alto para predecir los valores de la superficie faltantes de casas y apartamentos en función de su precio, dormitorios, ubicación y tipo de negocio. 
#### Se imputarán los valores faltantes de superficie, calculando el valor del m**2 por barrio promedio con la data disponible.

In [327]:
barrios_civil = df_2[df_2["negocio"]=="Venta"]["barrio"].value_counts().index

In [328]:
venta_civil = df_2[df_2["negocio"]=="Venta"]

In [329]:
venta_civil["$/m"] = round(venta_civil["precio U$S"]/venta_civil["superficie"],3)

In [330]:
venta_civil.head(5)

,precio U$S,superficie,tipo,dormitorios,barrio,negocio,$/m
0,380000.0,130.0,Apartamento,3,Punta Carretas,Venta,2923.077
1,129000.0,75.0,Apartamento,2,Ciudad Vieja,Venta,1720.000
2,115000.0,30.0,Apartamento,1,Punta Carretas,Venta,3833.333
4,380000.0,249.0,Casa,4,Prado,Venta,1526.104
5,198900.0,86.0,Apartamento,2,Centro,Venta,2312.791


In [332]:
prom = []
for barrio in barrios_civil:
    prom.append(np.mean(venta_civil[venta_civil["barrio"]==barrio]["$/m"]))

In [333]:
valor_prom = dict(zip(barrios_civil,prom))

In [334]:
valor_prom

{'Pocitos': 2865.0989165180167,
 'Centro': 2249.020260553129,
 'Punta Carretas': 3140.672779710145,
 'Carrasco': 2201.7682649294247,
 'La Blanqueada': 2167.2854005201557,
 'Tres Cruces': 2325.7012309058614,
 'Parque Batlle': 2353.5339353049912,
 'Buceo': 2331.1698682926826,
 'Aguada': 2033.5174685863876,
 'Prado': 1755.3315027322403,
 'Palermo': 2334.5490489296635,
 'Punta Gorda': 2119.3801018867925,
 'Carrasco Norte': 1754.5345621621623,
 'Ciudad Vieja': 1758.9215918367347,
 'La Comercial': 1471.3685158730161,
 'Bella Vista': 1858.6160869565217,
 'Brazo Oriental': 1505.9960727272723,
 'Jacinto Vera': 1741.5101238095237,
 'Goes': 1605.0709629629632,
 'Reducto': 1474.2484102564101,
 'Atahualpa': 1559.9680746268657,
 'Aires Puros': 988.3588333333332,
 'Capurro': 1109.0703571428571,
 'Villa Dolores': 2291.534277777778,
 'Villa Muñoz': 1383.2356470588236,
 'La Figurita': 1385.956909090909,
 'Larrañaga': 2077.624125}

In [335]:
venta_civil[venta_civil["barrio"]=="Villa Dolores"]

,precio U$S,superficie,tipo,dormitorios,barrio,negocio,$/m
110,240000.0,130.0,Casa,3,Villa Dolores,Venta,1846.154
580,410000.0,559.0,Casa,3,Villa Dolores,Venta,733.453
621,61000.0,29.0,Apartamento,1,Villa Dolores,Venta,2103.448
1225,134000.0,50.0,Apartamento,1,Villa Dolores,Venta,2680.000
1393,119000.0,43.0,Apartamento,1,Villa Dolores,Venta,2767.442
1462,180000.0,81.0,Casa,3,Villa Dolores,Venta,2222.222
1796,170000.0,53.0,Apartamento,2,Villa Dolores,Venta,3207.547
3579,670000.0,180.0,Apartamento,3,Villa Dolores,Venta,3722.222
3710,159000.0,65.0,Apartamento,3,Villa Dolores,Venta,2446.154
4035,258000.0,139.0,Casa,3,Villa Dolores,Venta,1856.115


In [277]:
#Viendo a simple vista este diccionario, en el barrio "La Figurita" hay un valor extraño.
#Para evitar problemas de cálculo del m2:
#Se eliminará toda fila con superficie menor a 10m2 (número arbitrario)

In [331]:
venta_civil["$/m"].describe()

count    10894.000000
mean      2404.838350
std        951.646159
min          0.004000
25%       1828.927500
50%       2427.330000
75%       2982.020000
max      20625.000000
Name: $/m, dtype: float64